In [12]:
#import os
#import geopandas as gpd
#from geopandas import GeoSeries
#import pandas as pd
#from geocube.api.core import make_geocube
#import rasterio
#import numpy as np
#import sys

In [1]:
def calculate_geo(x, path):
    import os
    import geopandas as gpd
    from geopandas import GeoSeries
    import pandas as pd
    def getXY(pt):
        return (pt.x, pt.y)
    polygon = gpd.read_file(path+'\\'+x+"_parcel.geojson")
    points = gpd.read_file(path+'\\'+x+"_points.geojson")
    polygon = polygon.to_crs(4269)
    Parcel_Centroid = GeoSeries(polygon['geometry']).centroid
    Parcel_Centroid = GeoSeries(polygon['geometry']).centroid
    x,y = [list(t) for t in zip(*map(getXY, Parcel_Centroid))]
    polygon['Centroid_X'] = x
    polygon['Centroid_Y'] = y
    polygon = polygon.to_crs(5070)
    polygon["PARCEL_AREA"] = polygon['geometry'].area/ 10**6
    #there left table has to be a gdf and the right df HAS to be a simple df to result in a gdf
    state = polygon.merge(pd.DataFrame(points.drop(columns='geometry')), on='PRCLDMPID')
    return state

In [3]:
#path = #path to intermediate files
#polygon = gpd.read_file('parcel.geojson')
#points = gpd.read_file('points.geojson')

In [4]:
#polygon = polygon.to_crs(4269)

In [5]:
#Parcel_Centroid = GeoSeries(polygon['geometry']).centroid

<ipython-input-5-120b4ad3e5d7>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  Parcel_Centroid = GeoSeries(polygon['geometry']).centroid


In [6]:
#def getXY(pt):
    #return (pt.x, pt.y)
#Parcel_Centroid = GeoSeries(polygon['geometry']).centroid
#x,y = [list(t) for t in zip(*map(getXY, Parcel_Centroid))]
#polygon['Centroid_X'] = x
#polygon['Centroid_Y'] = y

<ipython-input-6-07187857b746>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  Parcel_Centroid = GeoSeries(polygon['geometry']).centroid


In [7]:
#polygon = polygon.to_crs(5070)

In [8]:
#polygon["Area_SQKM"] = polygon['geometry'].area/ 10**6

In [9]:
#there left table has to be a gdf and the right df HAS to be a simple df to result in a gdf
#state = polygon.merge(pd.DataFrame(points.drop(columns='geometry')), on='PRCLDMPID')